# Perform Lesion Network Mapping 'Overlap' Experiment

### Authors: Alexander Cohen, Christopher Lin

Notes:
- This notebook requires the NIMLAB Python 3 environment as a kernel. Directions at: (https://github.com/nimlab/software_env)
- This notebook is a combination of the old Simple Analysis Notebook and the overlap_maps notebooks (does the same thing).

In [ ]:
## Packages and environmental settings:

##Packages:
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
from matplotlib import pyplot as plt
from nilearn import image, plotting, regions
from nimlab import datasets as nimds
from nimlab import functions as nimf

%matplotlib inline

# Input/Output Setup

### 1. Provide your email address for future reference:

In [ ]:
# Enter your email address here in quotes:
creator_email = "alexander.cohen2@childrens.harvard.edu"

### 2. Where are your data? and where do you want the results to go?
NOTE: Provide a csv file created from the `Preprocessing` or `xnat_grabber` notebooks that points to your Lesions and fcMaps (will also work with older csv files from `xnat_gate`).

In [ ]:
# Specify the path to your dataset.csv file

dataset_csv = "your_dataset_csv_file.csv"
output_dir = "2_LNM_of_your_lesions_w_a_particular_connectome"

# EXAMPLE dataset_csv = "Prosopagnosia_w_Yeo1000.csv"
# EXAMPLE output_dir = "2_LNM_of_Prosopagnosia_w_Yeo1000"

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

img_csv = pd.read_csv(dataset_csv)
print("I found", len(img_csv), "participants:")
img_csv.head()

# Before we look at the *lesion network* overlap, let's review the *lesion* overlap map

In [ ]:
# Generate an "N-image" and a "Coverage Map", then show me:
lesions = [image.load_img(entry, dtype="float32") for entry in img_csv['roi_2mm']]
lesion_overlap = image.math_img("img*" + str(len(img_csv)), img=image.mean_img(lesions))
lesion_mask = image.math_img("img>=1", img=lesion_overlap)

plotting.plot_stat_map(
    lesion_overlap, display_mode="z", cut_coords=range(-54,72,6), cmap="Spectral_r", colorbar=True
)
plotting.plot_stat_map(
    lesion_mask, display_mode="z", cut_coords=range(-54,72,6), cmap="bwr", colorbar=True
)

# Lesion Network Mapping

### 1. Load the fcMap Tmaps and generate a plot relating T-thresholds to the maximum percentage of overlap
#### Select a brain mask for visualization
- Default brain mask is `MNI152_T1_2mm_brain_mask_dil`
- A more strict brain mask option available is `MNI152_T1_2mm_brain_mask`
- Additional options are listed in `nimfs.check_mask()` or you can specify a path to a binary mask of your choosing
    
NOTES:
- This can be useful for selecting an appropriate threshold.
- See [Cohen and Fox, 2020](https://doi.org/10.1093/brain/awaa095) for details.


In [ ]:
# Specify the range of T score levels to test as (start, end+1, increment).

t_levels = range(4,16,1)
# EXAMPLE t_levels = range(4,16,1)

# You can change this mask to a more restrictive one for visualization purposes
# Can be a name of a mask in nimlab.datasets or a path to a binary mask nifti file.
mask="MNI152_T1_2mm_brain_mask_dil"


t_maps = [image.load_img(entry) for entry in img_csv['t']]

max_positives = []
max_negatives = []
for t in tqdm(t_levels):
    pos_overlap, neg_overlap = nimf.get_pos_neg_overlap_maps(t_maps, t, mask = mask) 
    max_positives.append(pos_overlap.get_fdata().max())
    max_negatives.append(neg_overlap.get_fdata().max())

plt.plot(t_levels, max_positives, label='Positive')
plt.plot(t_levels, max_negatives, label='Negative')
plt.title("T-score levels vs. peak LNM overlap")
plt.xlabel("T-score threshold")
plt.ylabel("Max overlap")
plt.legend();

### 2. At a specific threshold, show me the spatial overlap at various percentages

NOTES:
- We have typically considered a rough guideline of $\geq$90-95% overlap to be important (100% is easiest to interpret of course)
- Be mindful of how you think about thresholds for the overlap:
  - Here, I have modified the code to show voxels **$\geq$ instead of $\gt$thresholds**, so if a voxel is present in exactly 75% of subjects, I am displaying it here.
  - FSLeyes (and nilearn) default to show data $\gt$threshold, so if you load the `LNM_fraction_xxx.nii.gz` and set the range to 0.75 to 1, you are viewing voxels **$\gt$0.75, not $\geq$0.75**.
  - `direction` defaults to `"twosided"`, which considers both positive and negative connectivity when calculating overlaps. You can also set `direction` to `"positive"` or `"negative"` to consider only positive or negative connectivity when calculating overlaps, respectively.
  - `mode` defaults to `"fraction"`, which plots the overlap maps as a fraction of the total number of subjects. You can also set `mode` to `"count"` to plot the overlap maps as a count of the number of subjects that overlap at any particular region. 


In [ ]:
# Specify a T-score threshold

T_level = 9
# EXAMPLE T_level = 9

# Specify a range of overlap fractions to examine
range_to_examine = np.linspace(0.75,1,6)

nimf.plot_overlap_map_to_screen(
    t_maps, T_level, range_to_examine, mask = mask,
    direction = "twosided",
    mode = "fraction"
)

### 3. Final required step: Write your overlap maps to files

NOTE: I put this in a separate cell, so you can re-run the prior cell multiple times to view the differences. Once you've selected the T-score thresholds you want to use, specify them below to write the overlap maps to files
- `direction` defaults to `"twosided"`, which considers both positive and negative connectivity when calculating overlaps. You can also set `direction` to `"positive"` or `"negative"` to consider only positive or negative connectivity when calculating overlaps, respectively.

In [ ]:
# Specify the specific T-score thresholds you want to print in the array here:
for t in [5, 7, 9]:
    print("Writing Files for a T>",str(t))
    nimf.write_overlap_map_to_file(
        t_maps, t, output_dir, mask = mask,
        direction = "twosided"
    )

### 4a. Optional: Generate Regions of Interest

NOTES: 
- You probably found that a particular T-score threshold and fractional overlap that appears to most cleanly define some regions of interest
- Put in those parameters here and then adjust the `minimum_region_size` parameter to isolate your ROIs.
- Once you have a set of ROIs that you like, set `write_to_file` to `True` they will be written to files

In [ ]:
# Copy your favorite parameters from above:
T_level = 9
fractional_overlap = 0.95

# This is in mm^3, so each 2x2x2mm voxel is 8mm^3, anywhere from 100 to 250 seems good, if your regions are tiny, you will have to shrink this...
minimum_region_size=250

# Do you want to print out the regions as nifti files:
write_to_file = True


# Isolate the regions into separate ROIs:
peak_regions, roi_index = nimf.define_ROIs_from_overlap(t_maps, T_level, fractional_overlap, minimum_region_size, mask = mask)

for i in range(0,len(roi_index)):
    plotting.plot_stat_map(image.index_img(peak_regions,i),
                           colorbar=False,title="ROI #" + str(i+1),cmap="bwr")
    
if write_to_file:
    for i in range(0,len(roi_index)):
        filename = os.path.join(output_dir,
                         "LNM_ROI_#" + str(i+1)
                         + "_at_T-" + str(T_level)
                         + "_present_in_" + str(fractional_overlap)
                         + "_of_" + str(len(img_csv)) + "_subjects.nii.gz")
        print("Writing",filename)
        image.math_img("img!=0",img=image.index_img(peak_regions,i)).to_filename(filename)